In [1]:
import pandas as pd
import numpy as np

### Part One: Load Data

In [2]:
cell_phones = pd.read_csv('https://raw.githubusercontent.com/zalihat/cell_phone_dataset/master/cell_phones.csv')
population = pd.read_csv('https://raw.githubusercontent.com/zalihat/cell_phone_dataset/master/population1.csv')
geo_country_codes = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv')

In [3]:
display(cell_phones.head(), cell_phones.shape)
display(population.head(), population.shape)
display(geo_country_codes.head(), geo_country_codes.shape)

,Country,year,cell_phones_total
0,Afghanistan,1960,0.0
1,Afghanistan,1961,0.0
2,Afghanistan,1962,0.0
3,Afghanistan,1963,0.0
4,Afghanistan,1964,0.0


(11640, 3)

,Country,year,population
0,Afghanistan,1800,3280000
1,Afghanistan,1801,3280000
2,Afghanistan,1802,3280000
3,Afghanistan,1803,3280000
4,Afghanistan,1804,3280000


(58695, 3)

,country,g77_and_oecd_countries,income_3groups,income_groups,is--country,iso3166_1_alpha2,iso3166_1_alpha3,iso3166_1_numeric,iso3166_2,landlocked,...,longitude,main_religion_2008,name,un_sdg_ldc,un_sdg_region,un_state,unicef_region,unicode_region_subtag,world_4region,world_6region
0,abkh,others,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Abkhazia,NaN,NaN,False,NaN,NaN,europe,europe_central_asia
1,abw,others,high_income,high_income,True,AW,ABW,533.0,NaN,coastline,...,-69.96667,christian,Aruba,un_not_least_developed,un_latin_america_and_the_caribbean,False,NaN,AW,americas,america
2,afg,g77,low_income,low_income,True,AF,AFG,4.0,NaN,landlocked,...,66.00000,muslim,Afghanistan,un_least_developed,un_central_and_southern_asia,True,sa,AF,asia,south_asia
3,ago,g77,middle_income,lower_middle_income,True,AO,AGO,24.0,NaN,coastline,...,18.50000,christian,Angola,un_least_developed,un_sub_saharan_africa,True,ssa,AO,africa,sub_saharan_africa
4,aia,others,NaN,NaN,True,AI,AIA,660.0,NaN,coastline,...,-63.05000,christian,Anguilla,un_not_least_developed,un_latin_america_and_the_caribbean,False,NaN,AI,americas,america


(273, 21)

### Part Two: Join Data

In [4]:
# merging cell_phones and population dataset
merged_data = cell_phones.merge(population, on = ['Country', 'year'], how = 'inner')
print(merged_data.shape)
display(merged_data.head())

# renaming geo_country_code columns
geo_country_codes.rename(columns={'country': 'geo', 'name': 'Country'}, inplace=True)

selected_geo = geo_country_codes[['geo', 'Country']]

# merging selected_geo and merged_data
md = merged_data.merge(selected_geo, on = 'Country', how = 'inner')
print(md.shape)
display(md.tail())


(11640, 4)


,Country,year,cell_phones_total,population
0,Afghanistan,1960,0.0,9000000
1,Afghanistan,1961,0.0,9170000
2,Afghanistan,1962,0.0,9350000
3,Afghanistan,1963,0.0,9540000
4,Afghanistan,1964,0.0,9740000


(11640, 5)


,Country,year,cell_phones_total,population,geo
11635,Zimbabwe,2015,12800000.0,13800000,zwe
11636,Zimbabwe,2016,12900000.0,14000000,zwe
11637,Zimbabwe,2017,14100000.0,14200000,zwe
11638,Zimbabwe,2018,12900000.0,14400000,zwe
11639,Zimbabwe,2019,13200000.0,14600000,zwe


### Part Three: Make Features

In [5]:
md['cellphones_per_person'] = md.apply(lambda x: x['cell_phones_total']/x['population'], axis = 1)
display(md[(md['year'] == 2017) & (md['Country'] == 'United States')])

,Country,year,cell_phones_total,population,geo,cellphones_per_person
11157,United States,2017,400000000.0,325000000,usa,1.230769


In [6]:
# modifying the geo column
md['geo'] = md.geo.apply(lambda x: x.upper())

display(md.head())

,Country,year,cell_phones_total,population,geo,cellphones_per_person
0,Afghanistan,1960,0.0,9000000,AFG,0.0
1,Afghanistan,1961,0.0,9170000,AFG,0.0
2,Afghanistan,1962,0.0,9350000,AFG,0.0
3,Afghanistan,1963,0.0,9540000,AFG,0.0
4,Afghanistan,1964,0.0,9740000,AFG,0.0


### Part Four: Process Data

In [7]:
#describing numeric columns
display(md.describe())

#describing categorical columns
display(md.describe(include=[object]))

,year,cell_phones_total,population,cellphones_per_person
count,11640.000000,1.164000e+04,1.164000e+04,11640.000000
mean,1989.500000,7.875722e+06,2.710307e+07,0.235536
std,17.318846,5.572496e+07,1.086187e+08,0.433303
min,1960.000000,0.000000e+00,4.380000e+03,0.000000
25%,1974.750000,0.000000e+00,1.117500e+06,0.000000
50%,1989.500000,0.000000e+00,5.100000e+06,0.000000
75%,2004.250000,5.615000e+05,1.590000e+07,0.233667
max,2019.000000,1.730000e+09,1.430000e+09,2.126068


,Country,geo
count,11640,11640
unique,194,194
top,Saudi Arabia,COG
freq,60,60


In [8]:
# Top 5 countries with the most cell phones
pd.set_option('float_format', '{:f}'.format)
top5 = md.groupby(['Country', 'year']).cell_phones_total.sum().reset_index()
top5_countries = top5[top5.year == 2017].sort_values(by='cell_phones_total', ascending = False)[:5]
display(top5_countries.drop(columns = ['year']))

,Country,cell_phones_total
2157,China,1470000000.000000
4617,India,1170000000.000000
4677,Indonesia,435000000.000000
11157,United States,400000000.000000
8577,Russia,227000000.000000
